In [ ]:
# %load_ext autoreload
# %autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
cfgList=[]
filesDic={}
files2ptKey={}
files3ptKey={}

data={}

def loadData(basePath):
    f={}
    for key in filesDic.keys():
        f[key]={}
        for file in filesDic[key]:
            f[key][file] = h5py.File(basePath + file + '.h5','r')

    for key in files2ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for projg in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][projg]={}
                        for cont in tF[projAStr][projBStr][projg][cfgList[0]].keys():
                            data[key][file][projA][projB][projg][cont]=np.array([
                                f[key][file][projAStr][projBStr][projg][cfg][cont][()] for cfg in cfgList
                                ])

    for key in files3ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for projg in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][projg]={}
                        for cont in tF[projAStr][projBStr][projg][cfgList[0]].keys():
                            data[key][file][projA][projB][projg][cont]=np.array([
                                f[key][file][projAStr][projBStr][projg][cfg][cont][()] for cfg in cfgList
                                ])

    for key in filesDic.keys():
        for file in filesDic[key]:
            f[key][file].close()

path_projData='/project/s1174/lyan/code/projectData/'

basePath_NJN=path_projData+'nucleon_sigma_term/cA211.53.24/NJN/proj/'
basePath_NJNpi=path_projData+'nucleon_sigma_term/cA211.53.24/NJNpi_GEVP/proj/'
basePath_loop=path_projData+'nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/data_post_merged/'

with h5py.File(basePath_NJN+'NJN.h5') as f:
    cfgList_NJN=list(yu.deepKey(f,3).keys())

with h5py.File(basePath_NJNpi+'N_2pt.h5') as f:
    cfgList_NJNpi=list(yu.deepKey(f,3).keys())

cfgList=list( set(cfgList_NJN) & set(cfgList_NJNpi) )
# cfgList=cfgList_NJNpi

cfgList.sort()
Ncfg=len(cfgList)
print(Ncfg)


filesDic={}
files2ptKey=[]
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['NJN']
filesDic['NJNpi']=['NJN-pi0i']
loadData(basePath_NJN)

filesDic={}
files2ptKey=['NN','NNpi','NpiN','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T','N_2pt-pi0i']
filesDic['NpiN']=['D1ii','N_2pt-pi0f']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt','N_2pt-pi0f-pi0i','T-pi0f','D1ii-pi0i']
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['N_2pt-j']
filesDic['NJNpi']=['B','W','Z','N_2pt-j-pi0i','N_2pt-j&pi0i','T-j']
loadData(basePath_NJNpi)

# filesDic={}
# files2ptKey=['NN','NNpi','NpiN','NpiNpi']
# filesDic['NN']=['N_2pt']
# filesDic['NNpi']=['T']
# filesDic['NpiN']=['D1ii']
# filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
# files3ptKey=['NJNpi']
# filesDic['NJNpi']=['B','W','Z']
# loadData(basePath_NJNpi)


data['others']={}
with h5py.File(basePath_loop+'insertLoop.h5') as f:
    data['others']['u+d_id_vev']=np.array([f[cfg]['u+d_id_vev'][()] for cfg in cfgList])
    data['others']['u-d_g5_vev']=np.array([f[cfg]['u-d_g5_vev'][()] for cfg in cfgList])
with h5py.File(basePath_loop+'pi0Loop.h5') as f:
    data['others']['pi0Loop_g5_vev']=np.array([f[cfg]['pi0Loop_g5_vev'][()] for cfg in cfgList])

# getDat

In [ ]:
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1

diags_key={}
diags_all=set(); diags_pi0Loop=set(); diags_jLoop=set()
for key in data.keys():
    if key in ['others']:
        continue
    diags_key[key] = set(data[key])
    diags_all = diags_all | diags_key[key]
    for diag in data[key]:
        t=diag.split('-')
        if 'pi0i' in t or 'pi0f' in t:
            diags_pi0Loop.add(diag)
        if 'j' in t:
            diags_jLoop.add(diag)
diags_loopful = diags_pi0Loop | diags_jLoop
diags_loopless = diags_all - diags_loopful
diags_jLoopless = diags_all - diags_jLoop
diags_all_2pt = diags_key['NN'] | diags_key['NNpi'] | diags_key['NpiNpi']
diags_all_3pt = diags_all - diags_all_2pt

projgs_P012=['P0','P01','P02']
projgs_Pz12=['Pz','P01','P02']

Ntime=yu.deepKey(data['NN'],5).shape[1]

def getDat2pt(opa,opb,projg,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['12']:
        opa1=('n&pi+',proja); opa2=('p&pi0',proja)
        return np.sqrt(2/3)*getDat2pt(opa1,opb,projg,diags)-np.sqrt(1/3)*getDat2pt(opa2,opb,projg,diags)
    if tpb in ['12']:
        opb1=('n&pi+',projb); opb2=('p&pi0',projb)
        return np.sqrt(2/3)*getDat2pt(opa,opb1,projg,diags)-np.sqrt(1/3)*getDat2pt(opa,opb2,projg,diags)

    if tpa in ['p'] and tpb in ['p']:
        key='NN'
        return data[key]['N_2pt'][proja][projb][projg][tpa+'_'+tpb]+np.zeros([Ncfg,Ntime])
    elif tpa in ['p']:
        key='NNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['T']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0i']:
                return [tpa+'_p'] if tpb =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    elif tpb in ['p']:
        key='NpiN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['D1ii']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0f']:
                return ['p_'+tpb] if tpa =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return np.conj(test_NNpi)*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    else:
        key='NpiNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B_2pt','W_2pt','Z_2pt']:
                return [tpa+'_'+tpb]
            if diag in ['M_correct_2pt']:
                return [tpa+'_'+tpa] if tpa==tpb else []
            if diag in ['N_2pt-pi0f-pi0i']:
                return ['p_p'] if tpa==tpb=='p&pi0' else []
            if diag in ['T-pi0f']:
                return ['p_'+tpb] if tpa=='p&pi0' else []
            if diag in ['D1ii-pi0i']:
                return [tpa+'_p'] if tpb=='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NpiNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])

def getDat2ptMat(ops,projg,diags):
    t=np.transpose([[getDat2pt(opa,opb,projg,diags) for opb in ops] for opa in ops],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(opa,opb,Gc,iso,projg,tf,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['12']:
        opa1=('n&pi+',proja); opa2=('p&pi0',proja)
        return np.sqrt(2/3)*getDat3pt(opa1,opb,Gc,iso,projg,tf,diags)-np.sqrt(1/3)*getDat3pt(opa2,opb,Gc,iso,projg,tf,diags)
    if tpb in ['12']:
        opb1=('n&pi+',projb); opb2=('p&pi0',projb)
        return np.sqrt(2/3)*getDat3pt(opa,opb1,Gc,iso,projg,tf,diags)-np.sqrt(1/3)*getDat3pt(opa,opb2,Gc,iso,projg,tf,diags)

    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]

    if tpa in ['p'] and tpb in ['p']:
        key='NJN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['NJN','N_2pt-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont) for diag in t_diags for cont in get_conts(diag)]
        return np.sum([data[key][dc[0]][proja][projb][projg][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])       
    elif tpa in ['p']:
        key='NJNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B','W','Z']:
                return [tpa+'_j'+iso+'_'+tpb+'_deltat_'+str(tf)]
            if diag in ['N_2pt-j&pi0i','N_2pt-j-pi0i','NJN-pi0i']:
                return [tpa+'_p'+'_j'+iso+'_deltat_'+str(tf)] if tpb == 'p&pi0' else []
            if diag in ['T-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NJNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])
    elif tpb in ['p']:
        if projb[0] != (0,0,0):
            return np.zeros([Ncfg,tf-1])
        return np.conj(getDat3pt(opb,opa,Gc,iso,projg,tf,diags)[:,::-1])
    else:
        return np.zeros([Ncfg,tf-1])
    
def getDat3ptMat(opas,opbs,Gc,iso,projg,tf,diags):
    t=np.transpose([[getDat3pt(opa,opb,Gc,iso,projg,tf,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    return t

tfList=[10,12,14]
def getDat(opas,opbs,Gc,iso,projg,diags):
    pta=opas[0][1][0]; ptb=opbs[0][1][0]
    assert(pta==(0,0,0)); pc=ptb
    if pc==(0,0,0) and projg=='Pzt':
        projg='Pz'
    if projg in ['Pz','Pzt']:
        projg_2pt='P0'
    else:
        projg_2pt=projg
    dat2ptA=getDat2ptMat(opas,projg_2pt,diags)
    dat2ptB=getDat2ptMat(opbs,projg_2pt,diags)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(opas,opbs,Gc,iso,projg,tf,diags)
    if (Gc,iso) == ('id','+'):
        dat_jLoop=data['others']['u+d_id_vev']
    elif (Gc,iso) == ('g5','-'):
        dat_jLoop=data['others']['u-d_g5_vev']
    else:
        dat_jLoop=np.zeros(Ncfg)

    t=np.array([
        np.array([dat2ptA[cfg],dat2ptB[cfg]]+[np.array([dat3pt[tf][cfg] for tf in tfList],dtype=object)]+[dat_jLoop[cfg]],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    return t

# 2pt

In [ ]:
def run2pt(opa,opb,diags,title=None,ylim1=None,ylim2=[.4,1.1],preFactor=1,axs=None,avgConjFlag=False,xshift=0,projgs=['P01','P0','P02']):
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=opa[0]+'_'+opb[0]+(': '+title if title!=None else '')
        fig.suptitle(title)

    for projg in projgs:
        colorDic={'P01':'r','P02':'b','P0':'g'}
        
        dat=preFactor*getDat2pt(opa,opb,projg,diags)
        if avgConjFlag:
            dat=preFactor*(getDat2pt(opa,opb,projg,diags)+np.conj(getDat2pt(opb,opa,projg,diags)))/2

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=colorDic[projg],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))  
        if ylim1=='sym':
            t=max(abs(np.array(axs[irow,icol].get_ylim())))
            axs[irow,icol].set_ylim([-t,t])
        else:
            axs[irow,icol].set_ylim(ylim1)
        

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            t=np.log(t/np.roll(t,-1,axis=0))
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=colorDic[projg],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))
        axs[irow,icol].set_ylim(ylim2)
        if opa[1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif opa[1][0] == (0,0,1):
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange')
            

In [ ]:
# tests for N-N and N-Npi

pt=(0,0,0); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi1','a'))); op3=('p&pi0',(pt,('N1\\pi1','a')))
pt=(0,0,1); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi0','a'))); op3=('p&pi0',(pt,('N1\\pi0','a')))
# pt=(0,0,1); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N0\\pi1','a'))); op3=('p&pi0',(pt,('N0\\pi1','a')))

# N-N
# diags=diags_all
# (opa,opb)=(op1,op1)
# run2pt(opa,opb,diags)

# N-Npi vs. Npi-N
# diags=diags_all
# (opa,opb)=(op1,op2)
# run2pt(opa,opb,diags,preFactor=1,title='real',ylim1='sym')
# run2pt(opb,opa,diags,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag')
# run2pt(opb,opa,diags,preFactor=-1j,title='imag')
# (opa,opb)=(op1,op3)
# run2pt(opa,opb,diags,preFactor=1,title='real',ylim1='sym')
# run2pt(opb,opa,diags,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag')
# run2pt(opb,opa,diags,preFactor=-1j,title='imag')

# N-Npi: real imag, loop
# diags=diags_all
# (opa,opb)=(op1,op2)
# run2pt(opa,opb,diags,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag')
# (opa,opb)=(op1,op3)
# run2pt(opa,opb,diags,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag')
# diags=diags_loopless
# run2pt(opa,opb,diags,preFactor=1,title='real, loopless')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag, loopless')
# diags=diags_loopful
# run2pt(opa,opb,diags,preFactor=1,title='real, loopful')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag, loopful')

# N-Npi: avg N-pi0i and N-pi0f
# diags={'N_2pt-pi0i','N_2pt-pi0f'}
# (opa,opb)=(op1,op3)
# run2pt(opa,opb,diags,preFactor=-1j)
# run2pt(opb,opa,diags,preFactor=-1j)
# run2pt(opa,opb,diags,preFactor=-1j,avgConjFlag=True)

# N-Npi: test isospin 
diags=diags_all
fig,axs=yu.getFigAxs(1,2,8,10); fig.suptitle('real')
(opa,opb)=(op1,op2)
run2pt(opa,opb,diags,preFactor=1,axs=axs)
(opa,opb)=(op1,op3)
run2pt(opa,opb,diags,preFactor=1*(-1*np.sqrt(2)),axs=axs,xshift=0.5,avgConjFlag=True,ylim1='sym')

fig,axs=yu.getFigAxs(1,2,8,10); fig.suptitle('imag')
(opa,opb)=(op1,op2)
run2pt(opa,opb,diags,preFactor=-1j,axs=axs)
(opa,opb)=(op1,op3)
run2pt(opa,opb,diags,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=0.5,avgConjFlag=True)
diags=diags_loopless
(opa,opb)=(op1,op3)
run2pt(opa,opb,diags,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=-0.5,avgConjFlag=True)


In [ ]:
# tests for Npi-Npi

pt=(0,0,0); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi1','a'))); op3=('p&pi0',(pt,('N1\\pi1','a')))
pt=(0,0,1); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi0','a'))); op3=('p&pi0',(pt,('N1\\pi0','a')))

# BWZ vs M vs loops
# (opa,opb)=(op2,op2)
(opa,opb)=(op3,op3)
# (opa,opb)=(op2,op3)
run2pt(opa,opb,diags_all,title='all')
run2pt(opa,opb,{'B_2pt','W_2pt','Z_2pt'},title='BWZ')
run2pt(opa,opb,{'M_correct_2pt'},title='M')
run2pt(opa,opb,{'T-pi0f','D1ii-pi0i'},title='1Loop')
run2pt(opa,opb,{'N_2pt-pi0f-pi0i'},title='2Loops')

# c.c check
# diags=diags_all
# (opa,opb)=(op2,op3)
# run2pt(opa,opb,diags,title='real')
# run2pt(opb,opa,diags,title='real')
# run2pt(opa,opb,diags,preFactor=-1j,title='imag',ylim1='sym')
# run2pt(opb,opa,diags,preFactor=-1j,title='imag',ylim1='sym')

# isospin
diags=diags_all
# diags=diags_loopless
fig,axs=yu.getFigAxs(1,2,8,10); axs[0,0].set_ylim([0,0.001e-7])
(opa,opb)=(op2,op2)
run2pt(opa,opb,diags,preFactor=1,axs=axs,xshift=0)
(opa,opb)=(op3,op3)
run2pt(opa,opb,diags,preFactor=2,axs=axs,xshift=-0.5)
(opa,opb)=(op2,op3)
run2pt(opa,opb,diags,preFactor=(-1*np.sqrt(2)),axs=axs,xshift=0.5)

# 2pt GEVP

In [ ]:
# for report

def run2ptGEVP(ops,diags,title=None,ylim1=[.4,1.1],ylim2=[1e-4,1.1],axs=None,isospinCheck=False,xshift=0,projgs=['P01']):
    N_op=len(ops); tRef=4
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=', '.join([op[0] for op in ops]) + (': '+title if title!=None else '')
        # fig.suptitle(title)

    yu.addColHeader(axs,['Effective mass from E-values','Ratio from E-vectors'])
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}
    fmtDic={0:'s',1:'d',2:'o'}

    for projg in projgs:
        xshift_internal=0 if len(projgs)==1 else {'P0':0,'P01':-0.3,'P02':0.3}[projg]
        xshift_internal+=xshift
        dat=getDat2ptMat(ops,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            print(['red','green'][i])
            for j in range(len(tMean)):
                print(tmin+j,tMean[j],tErr[j])

            axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt='s',color=colorDic[i],elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))
        axs[irow,icol].set_ylim(ylim1)

        if ops[0][1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif ops[0][1][0] == (0,0,1):
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange') 

        icol=1
        for j in range(1):
            if isospinCheck and j!=0:
                continue
            def tFunc(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                t=t/t[0,:]
                return np.abs(t)
                return np.array([np.real(t[0]),np.imag(t[1]),np.imag(t[2])])
            (mean,cov,err)=yu.jackknife(dat,tFunc)
            mean0=mean[j,tmin:tmax]
            print(mean0)
        
            for i in range(2):
                ttmax=tmax
                tMean=mean[i,tmin:ttmax]/mean0;tErr=err[i,tmin:ttmax]/mean0

                if isospinCheck:
                    t={'p':1,'n&pi+':np.sqrt(2/3),'p&pi0':np.sqrt(1/3)}[ops[i][0]]
                    tMean/=t; tErr/=t
                    
                # temp_min=8
                # t_factor=1 if i!=1 else 1/np.sqrt(2)
                # print(i)
                # print(yu.npRound(tMean[temp_min:]*t_factor,3))
                # print(yu.npRound(tErr[temp_min:]*t_factor,3))

                print(['square','circle'][i])
                for t_j in range(len(tMean)):
                    print(tmin+t_j,tMean[t_j],tErr[t_j])

                axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt=fmtDic[i],color=colorDic[j],elinewidth=2,capsize=6,mfc='white')
            
            axs[irow,icol].set_xticks(range(0,tmax,1))
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])

pt=(0,0,0); op1=('p',(pt,'a')); op2=('12',(pt,('N1\\pi1','a')))

diags=diags_loopless
diags=diags_all
ops=[op1,op2]
run2ptGEVP(ops,diags)

In [ ]:
def run2ptGEVP(ops,diags,title=None,ylim1=[.4,1.1],ylim2=[1e-4,1.1],axs=None,isospinCheck=False,xshift=0,projgs=['P01']):
    N_op=len(ops); tRef=4
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=', '.join([op[0] for op in ops]) + (': '+title if title!=None else '')
        fig.suptitle(title)

    yu.addColHeader(axs,['Effective mass from E-values','Ratio from E-vectors'])
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}
    fmtDic={0:'s',1:'d',2:'o'}

    for projg in projgs:
        xshift_internal=0 if len(projgs)==1 else {'P0':0,'P01':-0.3,'P02':0.3}[projg]
        xshift_internal+=xshift
        dat=getDat2ptMat(ops,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt='s',color=colorDic[i],elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))
        axs[irow,icol].set_ylim(ylim1)

        if ops[0][1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif ops[0][1][0] == (0,0,1):
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange') 

        icol=1
        for j in range(N_op):
            if isospinCheck and j!=0:
                continue
            def tFunc(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                return np.abs(t)
                return np.array([np.real(t[0]),np.imag(t[1]),np.imag(t[2])])
            (mean,cov,err)=yu.jackknife(dat,tFunc)
            mean0=mean[j,tmin:tmax]
        
            for i in range(N_op):
                ttmax=tmax
                tMean=mean[i,tmin:ttmax]/mean0;tErr=err[i,tmin:ttmax]/mean0

                if isospinCheck:
                    t={'p':1,'n&pi+':np.sqrt(2/3),'p&pi0':np.sqrt(1/3)}[ops[i][0]]
                    tMean/=t; tErr/=t
                    
                # temp_min=8
                # t_factor=1 if i!=1 else 1/np.sqrt(2)
                # print(i)
                # print(yu.npRound(tMean[temp_min:]*t_factor,3))
                # print(yu.npRound(tErr[temp_min:]*t_factor,3))

                axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt=fmtDic[i],color=colorDic[j],elinewidth=2,capsize=6,mfc='white')
            
            axs[irow,icol].set_xticks(range(0,tmax,1))
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])

In [ ]:
pt=(0,0,0); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi1','a'))); op3=('p&pi0',(pt,('N1\\pi1','a')))
# pt=(0,0,1); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi0','a'))); op3=('p&pi0',(pt,('N1\\pi0','a')))

diags=diags_loopless
diags=diags_all
ops=[op1,op2]
run2ptGEVP(ops,diags)
ops=[op1,op3]
run2ptGEVP(ops,diags)
ops=[op1,op2,op3]
run2ptGEVP(ops,diags)
# run2ptGEVP(ops,diags,isospinCheck=True)

# test GEVP 2states vs 3states
# diags=diags_all
# fig,axs=yu.getFigAxs(1,2,10,10)
# ops=[op1]
# run2ptGEVP(ops,diags,axs=axs,xshift=+0.5)
# ops=[op1,op2]
# run2ptGEVP(ops,diags,axs=axs,xshift=+0.3)
# ops=[op1,op3]
# run2ptGEVP(ops,diags,axs=axs,xshift=-0.3)
# ops=[op1,op2,op3]
# run2ptGEVP(ops,diags,axs=axs)

In [ ]:
# pt=(0,0,0); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi1','a'))); op3=('p&pi0',(pt,('N1\\pi1','a')))
pt=(0,0,1); op1=('p',(pt,'a')); op2=('12',(pt,('N1\\pi0','a'))); op3=('p&pi0',(pt,('N0\\pi1','a')))


diags=diags_all
projgs=projgs_P012
projgs=['P01']
ops=[op1,op2]
run2ptGEVP(ops,diags,projgs=projgs)
ops=[op1,op3]
run2ptGEVP(ops,diags,projgs=projgs)
ops=[op1,op2,op3]
run2ptGEVP(ops,diags,projgs=projgs)

In [ ]:
# pt=(0,0,0); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi1','a'))); op3=('p&pi0',(pt,('N1\\pi1','a')))
pt=(0,0,1); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi0','a'))); op3=('p&pi0',(pt,('N1\\pi0','a')))


diags=diags_all
projgs=projgs_P012
ops=[op1,op2]
run2ptGEVP(ops,diags,projgs=projgs)
ops=[op1,op3]
run2ptGEVP(ops,diags,projgs=projgs)
ops=[op1,op2,op3]
run2ptGEVP(ops,diags,projgs=projgs)
# run2ptGEVP(ops,diags,isospinCheck=True)

In [ ]:
# # pt=(0,0,0); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N1\\pi1','a'))); op3=('p&pi0',(pt,('N1\\pi1','a')))
# pt=(0,0,1); op1=('p',(pt,'a')); op2=('n&pi+',(pt,('N0\\pi1','a'))); op3=('p&pi0',(pt,('N0\\pi1','a')))


# diags=diags_all
# projgs=projgs_P012
# ops=[op1,op2]
# run2ptGEVP(ops,diags,projgs=projgs)
# ops=[op1,op3]
# run2ptGEVP(ops,diags,projgs=projgs)
# ops=[op1,op2,op3]
# run2ptGEVP(ops,diags,projgs=projgs)

# 3pt

In [ ]:
def run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None,color='r'):
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+(': '+title if title!=None else ''))

    # yu.addColHeader(axs,['3pt'])

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        dat=getDat([opa],[opb],Gc,iso,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.array([np.real(preFactor*flip*(t[2][i_tf][:,0,0])) for i_tf in range(len(tfList))],dtype=object)
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color=color,elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(-6,6,1))
        axs[irow,icol].set_ylim(ylim1)

        # irow=0; icol=1
        # def tFunc(dat):
        #     t=np.mean(dat,axis=0)
            
        #     def getRes(i_tf):
        #         tf=tfList[i_tf]
        #         nom=t[2][i_tf][:,0,0]
        #         denom=np.sqrt( np.real(t[0][tf]) * np.real(t[1][tf]) *\
        #         np.real(t[0][1:tf])[::-1]/np.real(t[0][1:tf]) *\
        #         np.real(t[1][1:tf]/np.real(t[1][1:tf])[::-1]) )[:,0,0]
        #         return np.real(preFactor * (nom/denom-vevFactor*t[3]) )
        #     t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
        #     return t

        # (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        # colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        # fmtDic={10:'s',12:'d',14:'o'}

        # for i_tf in range(len(tfList)):
        #     tf=tfList[i_tf]
        #     tMean=mean[i_tf];tErr=err[i_tf]
        #     axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
        #         fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(-6,6,2))
        # axs[irow,icol].set_ylim(ylim2)

# pt1=(0,0,0); pt2=(0,0,1)
# proj1=(pt1,'a'); proj12=(pt2,'a')
# projg=(pt1,('N1\\pi1','a')); projg2=(pt2,('N1\\pi0','a')); projg3=(pt2,('N0\\pi1','a'))
# op1=('p',proj1); op12=('p',proj12)
# op2=('n&pi+',projg); op22=('n&pi+',projg2); op23=('n&pi+',projg3)
# op3=('p&pi0',projg); op32=('p&pi0',projg2); op33=('p&pi0',projg3)

In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('id','+',projgs_P012,1)
# (Gc,iso,projg,preFactor)=('gt','+','P0',1)
# (Gc,iso,projg,preFactor)=('gt','-','P0',1)

diags=diags_jLoopless
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
# (opa,opb)=(op01,op01)
# run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,axs=axs)
(opa,opb)=(op01,op02)
run3pt(opa,opb,Gc,iso,projgs,1j*preFactor,diags,axs=axs,color='g')
(opa,opb)=(op01,op03)
run3pt(opa,opb,Gc,iso,projgs,1j*preFactor*(-np.sqrt(2)),diags,axs=axs,color='b')

diags=diags_jLoopless
# diags=diags_loopless
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
# (opa,opb)=(op01,op11)
# run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,axs=axs)
(opa,opb)=(op01,op121)
run3pt(opa,opb,Gc,iso,projgs,-1j*preFactor,diags,axs=axs,color='g')
(opa,opb)=(op01,op131)
run3pt(opa,opb,Gc,iso,projgs,-1j*preFactor*(-np.sqrt(2)),diags,axs=axs,color='b')


In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)

diags=diags_jLoopless # | {'N_2pt-j'}
# diags=diags_all
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
# (opa,opb)=(op01,op11)
# run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,axs=axs)
# (opa,opb)=(op01,op121)
# run3pt(opa,opb,Gc,iso,projgs,-1j*preFactor,diags,axs=axs,color='g')
# (opa,opb)=(op01,op131)
# run3pt(opa,opb,Gc,iso,projgs,-1j*preFactor*(-np.sqrt(2)),diags,axs=axs,color='b')
(opa,opb)=(op01,op131)
run3pt(opa,opb,Gc,iso,projgs,1j,diags,axs=axs,color='b')


# 3pt2ptGEVP

In [ ]:
def run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None):
    tRef=8; tfAvgMin=tRef+1; tfAvgMax=15
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        # fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+'; '+', '.join([op[0] for op in opas]) + '; '+', '.join([op[0] for op in opbs]) + (': '+title if title!=None else ''))

    
    vevFactor = 1 if 'N_2pt-j' in diags and opas[0]==opbs[0] else 0

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        dat=getDat([opa],[opb],Gc,iso,projg,diags)

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[1],tRef)

            def get_eVec(eVecs):
                t=eVecs[:,0,0]
                t=eVecs[:,:,0]/t[:,None]
                return np.mean(t[tfAvgMin:tfAvgMax,:],axis=0)
            
            eVeca=get_eVec(eVecsa)
            eVecb=get_eVec(eVecsb)

            def getRes(i_tf):
                tf=tfList[i_tf]
                # eVeca=eVecsa[tf,:,0]
                # eVecb=eVecsb[tf,:,0]
                nom=np.conj(eVeca).T@t[2][i_tf]@eVecb
                denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                return np.real(preFactor*flip * (nom/denom-vevFactor*t[3]) )
                
            t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
            return t

        dat=getDat(opas[:1],opbs[:1],Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        print('Red: without GEVP improvement')

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print('source sink separation: ',tf)
            for t_t in range(len(np.arange(1 - tf//2,tf//2))):
                print(np.arange(1 - tf//2,tf//2)[t_t],tMean[t_t],tErr[t_t])
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')

        axs[irow,0].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim1)

        icol=2
        dat=getDat(opas,opbs,Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        print('Green: with GEVP improvement')

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print('source sink separation: ',tf)
            for t_t in range(len(np.arange(1 - tf//2,tf//2))):
                print(np.arange(1 - tf//2,tf//2)[t_t],tMean[t_t],tErr[t_t])
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim2)

# pt1=(0,0,0); pt2=(0,0,1)
# proj1=(pt1,'a'); proj12=(pt2,'a')
# projg=(pt1,('N1\\pi1','a')); projg2=(pt2,('N1\\pi0','a')); projg3=(pt2,('N0\\pi1','a'))
# op1=('p',proj1); op12=('p',proj12)
# op2=('n&pi+',projg); op22=('n&pi+',projg2); op23=('n&pi+',projg3)
# op3=('p&pi0',projg); op32=('p&pi0',projg2); op33=('p&pi0',projg3)

pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('12',(pt0,('N1\\pi1','a')))

(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)

diags=diags_jLoopless

opas=[op01]; opbs=[op01,op02]
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
def run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None):
    tRef=8; tfAvgMin=tRef+1; tfAvgMax=15
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,3,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+'; '+', '.join([op[0] for op in opas]) + '; '+', '.join([op[0] for op in opbs]) + (': '+title if title!=None else ''))

    
    vevFactor = 1 if 'N_2pt-j' in diags and opas[0]==opbs[0] else 0

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        dat=getDat([opa],[opb],Gc,iso,projg,diags)

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[1],tRef)

            def get_eVec(eVecs):
                t=eVecs[:,0,0]
                t=eVecs[:,:,0]/t[:,None]
                return np.mean(t[tfAvgMin:tfAvgMax,:],axis=0)
            
            eVeca=get_eVec(eVecsa)
            eVecb=get_eVec(eVecsb)

            def getRes(i_tf):
                tf=tfList[i_tf]
                # eVeca=eVecsa[tf,:,0]
                # eVecb=eVecsb[tf,:,0]
                nom=np.conj(eVeca).T@t[2][i_tf]@eVecb
                denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                return np.real(preFactor*flip * (nom/denom-vevFactor*t[3]) )
                
            t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
            return t

        dat=getDat(opas[:1],opbs[:1],Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')

        axs[irow,0].set_xticks(range(-6,6,1))
        axs[irow,icol].set_xticks(range(-6,6,1))
        axs[irow,icol].set_ylim(ylim1)

        icol=2
        dat=getDat(opas,opbs,Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(-6,6,1))
        axs[irow,icol].set_ylim(ylim2)

# pt1=(0,0,0); pt2=(0,0,1)
# proj1=(pt1,'a'); proj12=(pt2,'a')
# projg=(pt1,('N1\\pi1','a')); projg2=(pt2,('N1\\pi0','a')); projg3=(pt2,('N0\\pi1','a'))
# op1=('p',proj1); op12=('p',proj12)
# op2=('n&pi+',projg); op22=('n&pi+',projg2); op23=('n&pi+',projg3)
# op3=('p&pi0',projg); op32=('p&pi0',projg2); op33=('p&pi0',projg3)

In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)

diags=diags_jLoopless

opas=[op01]; opbs=[op01,op02]
test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)


opas=[op01]; opbs=[op01,op03]
test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('id','+',['P01'],1)

diags=diags_jLoopless
# diags=diags_loopless

opas=[op01]; opbs=[op11,op12]
test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

opas=[op01]; opbs=[op11,op13]
test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('id','+',projgs_P012,1)

diags=diags_jLoopless
# diags=diags_loopless

# opas=[op01]; opbs=[op11,op12]
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

opas=[op01]; opbs=[op11,op13]
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('g5gt','-',['P01'],1)

# diags=diags_all
diags=diags_jLoopless
# diags=diags_loopless
# diags=diags_all_2pt | {'N_2pt-j-pi0i'}

# opas=[op01]; opbs=[op11,op12]
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)


opas=[op01]; opbs=[op11,op131]
test_NJNpi=-1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('n&pi+',(pt0,('N1\\pi1','a'))); op03=('p&pi0',(pt0,('N1\\pi1','a')))
op11=('p',(pt1,'a')); op12=('n&pi+',(pt1,('N1\\pi0','a'))); op13=('p&pi0',(pt1,('N1\\pi0','a'))); op121=('n&pi+',(pt1,('N0\\pi1','a'))); op131=('p&pi0',(pt1,('N0\\pi1','a')))
op14=('12',(pt1,('N1\\pi0','a'))); op141=('12',(pt1,('N0\\pi1','a')))

(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)

# diags=diags_all
diags=diags_jLoopless
# diags=diags_loopless
# diags=diags_all_2pt | {'N_2pt-j-pi0i'}

# opas=[op01]; opbs=[op11,op12]
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)


opas=[op01]; opbs=[op11,op131,op141]
test_NJNpi=-1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)